Here we develop tools to extract code and meta-data from a notebook,
and to create a notebook from a *BrainCircuitAnalysis* instance.
We will start with the second goal.
For this we define a `BrainCircuitAnalysis` instance in this notebook, 
look at the *JSON* produced used as the backend data for this notebook,
and iteratively write code that will take a `BrainCircuitAnalysis` instance 
and produce a notebook that an analyst can use to tailor the analysis to their 
taste and run it.
After we can go from an analysis instance to a notebook,
we will explore how to start with a notebook and generate code for a 
`BrainCircuitAnalysis` instance.

In [10]:
#essential imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from dmt.model.interface import Interface
from dmt.tk.phenomenon import Phenomenon
from dmt.tk.parameters import Parameters
from dmt.tk.plotting import Bars, LinePlot, HeatMap
from dmt.tk.plotting.multi import MultiPlot
from neuro_dmt import terminology
from neuro_dmt.models.bluebrain.circuit.geometry import Cuboid
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition\
    import CompositionAnalysesSuite
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.connectome\
    import ConnectomeAnalysesSuite
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.tools\
    import PathwayMeasurement
from neuro_dmt.analysis.circuit import BrainCircuitAnalysis
from neuro_dmt.analysis.reporting import CircuitAnalysisReport

Below we present a mock-up of notebook cells as they should look once an
analysis is loaded.
This mock-up will be a mix of markup and code blocs,
which will all be loaded from the analysis instance.

## Introduction

A circuit model should reproduce experimentally measured cell densities.
For the mammalian cerebral cortex, the simplest and most frequent circuit
composition measurement is that of layer cell densities. We will use three 
such experimental measurements to validate brain circuits of the somatosensory
cortex.

## Methods

Cell density was measured in randomly sampled boxes of
dimension {}. Each sampled box was conditioned to lie in a
the spatial location specified by brain-region and layer.
The plot shows mean and standard-deviation of cell densities
sampled from {} cubes. Reference data plotted alongside model 
measurements was measured (somewhere) in the SSCx. Specific 
sub-region for the reference data is not known. all sub-regions 
in these plots show the same reference data

In [ ]:
class CellDensityAnalysis(BrainCircuitAnalysis):
    """
    Analyze cell density.
    """
    introduction = """
    A circuit model should reproduce experimentally measured cell densities.
    For the mammalian cerebral cortex, the simplest and most frequent circuit
    composition measurement is that of layer cell densities. We will use three
    such experimental measurements to validate brain circuits of the 
    somatosensory cortex.
    """
    methods = """
    Cell density was measured in randomly sampled boxes of dimension {}.
    Each sampled box was conditioned to lie in a the spatial location 
    specified by brain-region and layer. The plot shows mean and 
    standard-deviation of cell densities sampled from {} cubes. Reference data
    plotted alongside model measurements was measured (somewhere) in the SSCx.
    Specific sub-region for the reference data is not known. All sub-regions
    in these plots show the same reference data
    """
    
    phenomenon =\
        Phenomenon(
            "Cell Density",
            "Number of cells in a unit volume.",
            group="Composition ")
    
    class AdapterInterface(Interface):
        """
        Methods that the analysis expects an adapter to provide.
        """
        def random_position(self, circuit_model, **spatial_query):
            """
            A 3-d numpy.ndarray representing a random position in the 
            circuit's physical space corresponding to a spatial query.
            """
            raise NotImplementedError
        
        def get_cells(self, circuit_model, roi):
            """
            Cells in region of interest in the circuit's physical space.
            """
            raise NotImplementedError
            
        
    def sample_measurement(self, circuit_model, adapter, size_roi=50., **query):
        """
        Make a single measurement
        """
        spatial_query =\
            terminology.circuit.get_spatial_query(query)
        position =\
            adapter.random_position(
                circuit_model, **spatial_query)
    
        if position is None:
            return 0.
    
        cuboid_to_measure =\
            Cuboid(position - size_roi / 2.,
                   position + size_roi / 2.)
        cell_count =\
            adapter.get_cells(circuit_model, roi=cuboid_to_measure)\
                   .shape[0]
        spatial_volume =\
            cuboid_to_measure.volume
    
        return 1.e9 * cell_count / spatial_volume
    
    
    plotter = \
        MultiPlot(
            mvar="region",
            plotter=Bars(xvar="layer",
                         xlabel="Layer",
                         yvar="cell_density",
                         ylabel="Cell Density",
                         gvar="dataset"))
    report =\
        CircuitAnalysisReport

In [2]:
phenomenon =\
    Phenomenon(
        "Cell Density",
        "Number of cells in a unit volume.",
        group="Composition ")

In [3]:
class AdapterInterface(Interface):
    """
    Methods that the analysis expects an adapter to provide.
    """
    def random_position(self, circuit_model, **spatial_query):
        """
        A 3-d numpy.ndarray representing a random position in the 
        circuit's physical space corresponding to a spatial query.
        """
        raise NotImplementedError
        
    def get_cells(self, circuit_model, roi):
        """
        Cells in region of interest in the circuit's physical space.
        """
        raise NotImplementedError

In [6]:
def sample_measurement(circuit_model, adapter, size_roi=50., **query):
    """
    Make a single measurement
    """
    spatial_query =\
        terminology.circuit.get_spatial_query(query)
    position =\
        adapter.random_position(
            circuit_model, **spatial_query)
    
    if position is None:
        return 0.
    
    cuboid_to_measure =\
        Cuboid(position - size_roi / 2.,
               position + size_roi / 2.)
    cell_count =\
        adapter.get_cells(circuit_model, roi=cuboid_to_measure)\

               .shape[0]
    spatial_volume =\
        cuboid_to_measure.volume
    
    return 1.e9 * cell_count / spatial_volume

In [8]:
plotter = \
    MultiPlot(
        mvar="region",
        plotter=Bars(xvar="layer",
                     xlabel="Layer",
                     yvar="cell_density",
                     ylabel="Cell Density",
                     gvar="dataset"))

In [11]:
report =\
    CircuitAnalysisReport